In [5]:
import os

get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from src.chromaDB_setup import *
from src.github_parser import *
from src.embedding import *
import chromadb
from chromadb.config import Settings


In [7]:
repo = g.get_repo("vl4duu/vlad-mutilica.space")
# ("khoj-ai/khoj")

In [8]:
repo_files = get_repo_files("vl4duu/vlad-mutilica.space")

In [9]:
textual_chunks, code_chunks = chunk_repository_files(repo_files)

# Process textual chunks with ada-002
for doc in textual_chunks:
    textual_embedding = embed_textual_metadata(doc["content"])

# Process code chunks with CodeBERT
for doc in code_chunks:
    code_embedding = generate_code_embedding(doc["content"])



In [10]:
# Initialize the ChromaDB client with the new format
client = chromadb.Client(
    Settings(
        persist_directory="./chromaDB/dir1"  # Path for persistence
    )
)

In [11]:
# Create a collection for your repository data
collection = client.get_or_create_collection(name="repo_assistant")

In [12]:
# Add to ChromaDB
for idx, doc in enumerate(chunked_docs):
    collection.add(
        documents=[doc["content"]],
        metadatas=[{"file_name": doc["file_name"]}],
        ids=[f"{doc['file_name']}_{idx}"],
        embeddings=[embeddings[idx].tolist()]
    )

NameError: name 'chunked_docs' is not defined

In [ ]:
def retrieve_relevant_chunks(query, top_k=3):
    # Generate query embedding
    query_embedding = embed_textual_metadata(query).tolist()

    # Query ChromaDB
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )

    # Return relevant documents and metadata
    return results["documents"], results["metadatas"]

In [ ]:
query = "What is this repo supposed to do?"
retrieve_relevant_chunks(query)